# **Парсер данных**

Сейчас мы сделаем универсальный парсер. То есть тикер будет параметром, который вводится извне. Тайфрейм свечей тоже будет параметром. Даты тоже будут параметрами. Данные будем парсить с [ISS MOEX](https://iss.moex.com/iss/reference/), то есть напрямую с мосбиржжи.

Внизу можете видеть функции, которая принимает на вход дату, тикер и таймфрейм, а выдает датафрейм с ценой открытия, закрытия, хвостами свечи, объемом и временем.


Импортируем все нужные библиотеки

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from xml.etree import ElementTree as ET
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

# Парсер для акций



In [5]:
def stock_moex(date_from, ticker, timeframe):

  url = f"https://iss.moex.com/iss/engines/stock/markets/shares/securities/{ticker}/candles.json"
  params = {
      'from': date_from,
      'interval': timeframe,
      }
  response = requests.get(url, params=params)
  data = response.json()
  columns = data["candles"]["columns"]
  rows = data["candles"]["data"]
  df = pd.DataFrame(rows, columns=columns)

  df.columns = [f'open_{ticker}', f'close_{ticker}', f'high_{ticker}', f'low_{ticker}', f'value_{ticker}', f'volume_{ticker}', f'begin', f'end_{ticker}']
  df = df.set_index('begin')
  df = df.drop([f'value_{ticker}', f'volume_{ticker}', f'end_{ticker}'], axis=1)


  return df


In [6]:
stock_moex('2025-02-21', 'SBER', timeframe=1)

,open_SBER,close_SBER,high_SBER,low_SBER
begin,,,,
2025-02-21 06:59:00,315.48,315.48,315.48,315.48
2025-02-21 07:00:00,315.48,315.40,315.52,315.15
2025-02-21 07:01:00,315.45,315.27,315.45,315.20
2025-02-21 07:02:00,315.37,315.43,315.45,315.28
2025-02-21 07:03:00,315.45,315.90,315.90,315.45
...,...,...,...,...
2025-02-21 15:14:00,313.45,313.51,313.55,313.40
2025-02-21 15:15:00,313.54,313.30,313.54,313.21
2025-02-21 15:16:00,313.30,313.22,313.35,313.21


# Парсер для фьючерсов



In [7]:
def futures_moex(date_from, ticker, timeframe):

  url = f"https://iss.moex.com/iss/engines/futures/markets/forts/securities/{ticker}/candles.json"

  params = {
      'from': date_from,
      'interval': timeframe,
      }
  response = requests.get(url, params=params)
  response.raise_for_status()
  data = response.json()
  columns = data["candles"]["columns"]
  rows = data["candles"]["data"]
  df = pd.DataFrame(rows, columns=columns)

  df.columns = [f'open_{ticker}', f'close_{ticker}', f'high_{ticker}', f'low_{ticker}', f'value_{ticker}', f'volume_{ticker}', f'begin', f'end_{ticker}']
  df = df.set_index('begin')
  df = df.drop([f'value_{ticker}', f'volume_{ticker}', f'end_{ticker}'], axis=1)

  return df



In [8]:
futures_moex('2025-02-21', 'SiH5', timeframe=1)

,open_SiH5,close_SiH5,high_SiH5,low_SiH5
begin,,,,
2025-02-21 08:59:00,89598,89580,89598,89580
2025-02-21 09:00:00,89562,89570,89655,89535
2025-02-21 09:01:00,89572,89497,89572,89497
2025-02-21 09:02:00,89497,89442,89497,89439
2025-02-21 09:03:00,89456,89473,89549,89445
...,...,...,...,...
2025-02-21 17:19:00,90130,90086,90139,90064
2025-02-21 17:20:00,90082,90072,90099,90067
2025-02-21 17:21:00,90071,90096,90097,90070


# Итоговая функция

Давайте сделаем еще удобнее и напишем функцию, которой на вход будет поступать дата, таймфрейм и два тикера, а она будет выдавать красивые данные по свечам.

In [9]:
def parser_moex(data, stock_ticker, futures_ticker, timeframe):
    stock_df = stock_moex(data, stock_ticker, timeframe)
    futures_df = futures_moex(data, futures_ticker, timeframe)

    df = pd.merge(stock_df, futures_df, how='inner', on='begin')

    return df

In [10]:
parser_moex(data='2025-02-21', stock_ticker='SBER', futures_ticker='SiH5', timeframe=1)

,open_SBER,close_SBER,high_SBER,low_SBER,open_SiH5,close_SiH5,high_SiH5,low_SiH5
begin,,,,,,,,
2025-02-21 08:59:00,315.47,315.46,315.47,315.44,89598,89580,89598,89580
2025-02-21 09:00:00,315.47,315.47,315.47,315.21,89562,89570,89655,89535
2025-02-21 09:01:00,315.47,315.79,315.79,315.46,89572,89497,89572,89497
2025-02-21 09:02:00,315.79,315.57,315.79,315.57,89497,89442,89497,89439
2025-02-21 09:03:00,315.57,315.61,315.70,315.57,89456,89473,89549,89445
...,...,...,...,...,...,...,...,...
2025-02-21 15:14:00,313.45,313.51,313.55,313.40,90342,90302,90368,90278
2025-02-21 15:15:00,313.54,313.30,313.54,313.21,90302,90289,90307,90268
2025-02-21 15:16:00,313.30,313.22,313.35,313.21,90290,90318,90334,90285


Этой функцией мы и будем пользоваться